In [1]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.select import Select
import time
import pandas as pd
import os
from base import *

In [2]:
def my_test(a, b):
    return a + b
#bom['Value'] = bom.apply(bom, axis = 0, args = ("Part_Number", "QTY"))
#for qty, part_number in bom['Value']:
#    print(part_number, qty)
#a = pd.Series([bom["QTY"],bom["Part_Number"]])
#bom.columns()

In [4]:
board_qty = 10
IOC = "../source/Board.xlsx"
SHEETNAMEC = "Schematic1"
path = "D:\SPIDER\chromedriver.exe"
bom = pd.read_excel(io=IOC, sheetname=SHEETNAMEC)
bom.head(1)

exclude = pd.read_excel(io="../source/常规用贴片清单.xlsx", sheetname="Sheet1")

dest = pd.DataFrame()
material_name = "../dest/auto_process.xlsx"
standard_dest = pd.DataFrame()
standard_material_name = "../dest/标准件.xlsx"
custom_dest = pd.DataFrame()
custom_material_name = "../dest/定制件.xlsx"
other_dest = pd.DataFrame()
other_material_name = "../dest/已有物料.xlsx"

In [5]:
option = webdriver.ChromeOptions()
#option.add_extension('D:\SPIDER\LazyTestElementsExtractor.crx')
#option.add_argument('--user-data-dir=C:\Users\Administrator\AppData\Local\Google\Chrome\User Data')
browser=webdriver.Chrome(executable_path=path,chrome_options=option)
try:
    browser.get(web_location["home_page"])
    print("handle = ", browser.window_handles)
    wait = WebDriverWait(browser, 120)
    wait.until(lambda ele: ele.find_element_by_id(web_location["id_globalSearch"]))
    i = 1
    for index, columns in bom.iterrows():
        part_number, qty, need_part_qty, ref_designer = calc_need_qty(columns, board_qty)
        
        input = browser.find_element_by_id(web_location["id_globalSearch"])
        input.send_keys(part_number)
        input.send_keys(Keys.ENTER)
        check_store_click(browser, web_location["text_inquire_shenzhen"], wait)
        
        handle = browser.window_handles
        browser.switch_to_window(handle[1])
        part_number_shenzhen, making_shenzhen, on_road_shenzhen, on_store_shenzhen = store_find(browser, wait)
        browser.close()
        
        handle = browser.window_handles
        browser.switch_to_window(handle[0])
        check_store_click(browser, web_location["text_inquire_songgang"], wait)
        
        handle = browser.window_handles
        browser.switch_to_window(handle[1])
        part_number_songgang, making_songgang, on_road_songgang, on_store_songgang = store_find(browser, wait)
        browser.close()
        
        handle = browser.window_handles
        browser.switch_to_window(handle[0])
        
        part_number_c, description, supplier_num_str = add_specification(browser, wait)
        
        supplier_num = int(supplier_num_str[-6:-5])
        supplier = ""
        model = ""
        rohs = ""
        grades = ""
        for tr_num in range(1, 1+supplier_num):
            supplier_xpath = '//*[@id="report.projectlycle.list"]/tbody[2]/tr[' + str(tr_num) + ']/td[1]/div/span'
            model_xpath = '//*[@id="report.projectlycle.list"]/tbody[2]/tr[' + str(tr_num) + ']/td[3]/div/span'
            rohs_xpath = '//*[@id="report.projectlycle.list"]/tbody[2]/tr[' + str(tr_num) + ']/td[4]/div/span'
            kid_xpath = '//*[@id="report.projectlycle.list"]/tbody[2]/tr[' + str(tr_num) + ']/td[5]/div/span'
                         
            kid = (browser.find_element_by_xpath(kid_xpath).text)[0:1]
            if kid == "A" or kid == "B" or kid == "C":
                supplier += browser.find_element_by_xpath(supplier_xpath).text + ";"
                model += browser.find_element_by_xpath(model_xpath).text + ";"                                
                rohs += browser.find_element_by_xpath(rohs_xpath).text + ";"
            grades += kid + ";"

        print(str(i) + "   " + str(part_number) + "   " + str(description))
        if (part_number == part_number_shenzhen == part_number_c) and (part_number == part_number_songgang):
            if (on_store_songgang - making_songgang) > (need_part_qty+100):
                store_setted = "光伏松岗"
            elif (on_store_shenzhen - making_shenzhen) > (need_part_qty+100):
                store_setted = "深圳"           
            else:
                store_setted = "全无库存"
                
            series = pd.Series([part_number, description, 
                                model, supplier, "EACH", 
                                qty, ref_designer, need_part_qty,
                                making_shenzhen, on_road_shenzhen, on_store_shenzhen,
                                making_songgang, on_road_songgang, on_store_songgang,
                                grades, store_setted, rohs])
            
        else:
            series = pd.Series(["error"])
        #print(series)

        dest, standard_dest, custom_dest, other_dest = custom_standard_split(part_number, 
                                                                             dest, standard_dest, custom_dest, other_dest, 
                                                                              standard_made_list, custom_made_list, series, exclude)
       
        i += 1       
    saved_to_excel(dest, standard_dest, custom_dest, other_dest, 
                   material_name, standard_material_name, custom_material_name, other_material_name)
    dest

#finally:
#    pass
except Exception as e:
    saved_to_excel(dest, standard_dest, custom_dest, other_dest, 
                   material_name, standard_material_name, custom_material_name, other_material_name)
    print("invalid part number: \033[1;32m " + str(part_number) + "\033[0m" + str('\n') + str(e))

handle =  ['CDwindow-C29E3532472BECB8D1CAE85FC321866B']
1   20001-00003   片状电阻;1/10W-100Ω±1%-0603;RoHS
part number in exclude:  20001-00003
2   20001-00004   片状电阻;1/10W-1KΩ±1%-0603;RoHS
part number in exclude:  20001-00004
3   20001-00005   片状电阻;1/10W-10KΩ±1%-0603;RoHS
part number in exclude:  20001-00005
4   20001-00013   片状电阻;1/10;12k;±1%;0603;RoHS
5   20001-00017   片状电阻;1/10W-1.5KΩ±1%-0603;RoHS
6   20001-00018   片状电阻;1/10W-15KΩ±1%-0603;RoHS
part number in exclude:  20001-00018
7   20001-00025   片状电阻;1/10W-200Ω±1%-0603;RoHS
part number in exclude:  20001-00025
8   20001-00026   片状电阻;1/10W-2KΩ±1%-0603;RoHS
part number in exclude:  20001-00026
9   20001-00030   片状电阻;1/10W-2.2KΩ±1%-0603;RoHS
10   20001-00037   片状电阻;1/10W-3KΩ±1%-0603;RoHS
part number in exclude:  20001-00037
11   20001-00041   片状电阻;1/10W-3.3KΩ±1%-0603;RoHS
12   20001-00051   片状电阻;1/10W-4.7KΩ±1%-0603;RoHS
part number in exclude:  20001-00051
13   20001-00058   片状电阻;1/10W-510KΩ±1%-0603;RoHS
14   20001-00066   片状电阻;1/10W;6.

* **不能将pdm里没有的料号加载,否则会引发异常**
* **异常引发后要保存数据**

In [ ]:
'''
dest.to_excel("./auto_save_store.xlsx")

a = bom[["Part Number", "QTY"]]
for i, j in bom.iterrows():
    #print("i = ", i)
   # print("j = ", j)
    print(j["Part Number"])
    print(j["QTY"])
    
j
'''

In [ ]:
model

In [ ]:
s = "123,456"
print(s.replace(',', ''))

In [ ]:
"20001-00001" in list(exclude["Part Number"])

In [ ]:

#while(1):
print("1")